In [1]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import seaborn as sns # type: ignore
import matplotlib.pyplot as plt # type: ignore
import os
from pathlib import Path
from datetime import datetime

In [2]:
# eigene Utils
from utils.addTimeInformation import addTimeInformation
from utils.combineDataFrames import combineDataFrames
from utils.extraploation_class import Extrapolation
from utils.plotCalendarHeatmap import plotCalendarHeatmap
from utils.read_CSV import *
from utils.calcDifference import differenceBetweenDataframes
from utils.calculateConsumption import calculateConsumption, getConsumptionYear

In [3]:
# Faktoren, Variablen

consumption_development_rate = 1.05

# CSV einlesen (Erzeugung)

In [4]:
#alle Daten ab 2015 einlesen
directory_yearly_generation = getData("Erzeugung")

# Daten extrapolieren 2024 bis 2030
for year in range(2024,2031):
    prev_year_df =directory_yearly_generation.get(year-1).copy()    #Kopie des Dataframe des letzten Jahres
    extrapolated_data = Extrapolation(prev_year_df, year, 1.05, 1.03, 1.04)        #Erstellung eines neuen Objekts, mit einem DataFrame
    directory_yearly_generation[extrapolated_data.year]= extrapolated_data.df   #DataFrame in das Erzeugungsverzeichnis gespeichert wird


Data für 2015 loaded succsessfully.
Data für 2016 loaded succsessfully.
Data für 2017 loaded succsessfully.
Data für 2018 loaded succsessfully.
Data für 2019 loaded succsessfully.
Data für 2020 loaded succsessfully.
Data für 2021 loaded succsessfully.
Data für 2022 loaded succsessfully.
Data für 2023 loaded succsessfully.


# Erzeugung 2030 (Gesamt)

In [5]:
# Lade die Daten für das Jahr 2030
df_2030 = directory_yearly_generation.get(2030)

# Überprüfe, ob die Spalten vorhanden sind
required_columns = ['Wind Offshore', 'Wind Onshore', 'Photovoltaik']
if all(column in df_2030.columns for column in required_columns):
    # Berechne die Summe der gewünschten Spalten für jede 15-Minuten-Periode
    df_2030['Gesamterzeugung_EE'] = df_2030[required_columns].sum(axis=1)
    
    # Speichere die Ergebnisse in production_2030
    production_2030 = df_2030[['Datum', 'Gesamterzeugung_EE']]
    
    #print(production_2030)
else:
    print("Eine oder mehrere der erforderlichen Spalten fehlen im DataFrame für 2030.")

# Verbrauch 2030 (Gesamt)

In [6]:
consumption_existing_years = calculateConsumption(consumption_development_rate)
consumption_2030 = consumption_existing_years.get(2030)

Data für 2023 loaded succsessfully.


# Differenz bilden

In [7]:
# Annahme: consumption_2030 und production_2030 sind bereits definierte DataFrames mit einer 'Datum'-Spalte und einer 'Gesamtverbrauch'-Spalte

# Überprüfe, ob beide DataFrames die gleiche Zeitachse haben

resdidual_df = differenceBetweenDataframes(consumption_2030, production_2030)

print(resdidual_df)

                    Datum    Differenz
0     2030-01-01 00:00:00  2713.895385
1     2030-01-01 00:15:00  2511.901102
2     2030-01-01 00:30:00  2174.559249
3     2030-01-01 00:45:00  2393.623849
4     2030-01-01 01:00:00  2004.345863
...                   ...          ...
35035 2030-12-31 22:45:00  2675.650795
35036 2030-12-31 23:00:00  2662.133043
35037 2030-12-31 23:15:00  2531.624569
35038 2030-12-31 23:30:00  2330.997366
35039 2030-12-31 23:45:00  2143.091762

[35040 rows x 2 columns]


# CO2-Bilanz bilden

# User-Info